In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import re
import seaborn as sns

## Cargar base SIVIGILA346

In [ ]:
df_sivigila346 = pd.read_csv('../Github/dl-covid19-descriptive-reports/MinSalud/sivigila346/data/ExtraccionEv346.txt',
                             low_memory=False,
                             on_bad_lines='skip',
                             sep='|',
                             usecols = ['PersonaBasicaID',
                                        'Edad',
                                        'FechaDefuncion',
                                        'FechaHospitalizacion',
                                        'DepartamentoNotificacion',
                                        'FechaNotificacion'])

print('Número filas y columnas: ', df_sivigila346.shape)

In [ ]:
# Eliminar registros basura (son 2)
df_sivigila346 = df_sivigila346.dropna(subset=['FechaNotificacion'])
print('Número filas y columnas: ', df_sivigila346.shape)

## Solucionar registro de edad y Función de extracción de edades

In [ ]:
# Solucionar el formato de una edad, para que la función creada pueda ser utilizada
df_sivigila346['Edad'].mask(df_sivigila346['Edad'] == 'De 75años', 'De 75 años', inplace=True)

In [ ]:
# Función para extraer la edad en años
def extraer_edad(edad_str):
    match = re.search(r'\d+', edad_str)
    if not match:
        return None
    edad = int(match.group())
    
    if 'minuto' in edad_str or 'horas' in edad_str or 'día' in edad_str or 'días' in edad_str:
        return 0  # Menos de un año se considera 0 años
    elif 'meses' in edad_str or 'mes' in edad_str:
        return max(edad // 12, 0)  # Convertir meses a años
    return edad  # Edad en años

## Análisis para Bogotá

In [ ]:
# Seleccionar los datos de Bogotá
df_bog = df_sivigila346[df_sivigila346['DepartamentoNotificacion'] == 'Bogotá, D.C.'].copy()

# Eliminar registros con edad "No Definido" (son 11 para Bogotá)
df_bog = df_bog[df_bog['Edad'] != 'No Definido']

# Convertir la columna de edad a valores numéricos
df_bog['EdadAños'] = df_bog['Edad'].apply(extraer_edad)

# Agrupar edades
bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, np.inf]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df_bog['GrupoEdad'] = pd.cut(df_bog['EdadAños'], bins=bins, labels=labels, right=True)
df_bog.head()

In [ ]:
# Convertir las fechas a formato 'datetime'
df_bog['FechaNotificacion'] = pd.to_datetime(df_bog['FechaNotificacion'], format='%Y%m%d')
df_bog['FechaDefuncion'] = pd.to_datetime(df_bog['FechaDefuncion'], format='%Y%m%d')
df_bog['FechaHospitalizacion'] = pd.to_datetime(df_bog['FechaHospitalizacion'], format='%Y%m%d')

# Eliminar fechas de hospitalización erróneas
df_bog = df_bog[df_bog['FechaHospitalizacion'] != '19000101']

# Definir quien falleció o no estando hospitalizado
fecha_no_defuncion = pd.Timestamp('1900-01-01')
df_bog['FallecioHosp'] = df_bog['FechaDefuncion'] != fecha_no_defuncion

# Descartar los casos donde el ID es igual a 1
df_bog = df_bog[df_bog['PersonaBasicaID'] != '1']

# Definir olas COVID-19 (obtenidas de: https://github.com/TRACE-LAC/covid19-waves-bogota/blob/main/waves/outputs/waves.csv)
waves = {
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}


# Grafica 
plt.figure(figsize=(5, 5))
color=['#6b6ca3', '#87bcbd', '#6f9954', '#b1615c']
i = 0
for wave, (inicio, fin) in waves.items():
    df_wave = df_bog[(df_bog['FechaNotificacion'] >= inicio) & (df_bog['FechaNotificacion'] <= fin)]
    
    # Denominador: gente hospitalizada (fallecida o no) por Covid por grupo de edad
    total_registros = df_wave.groupby('GrupoEdad').size()
    
    # Numerador: gente que fallecio hospitalizada por Covid por grupo de edad
    fallecidosHosp = df_wave[df_wave['FallecioHosp']].groupby('GrupoEdad').size()
    
    # Calcular Case Fatality Ratio (CFR)
    HFR = (fallecidosHosp / total_registros).fillna(0)
    
    print(fallecidosHosp)
    print(total_registros)
    
    # Graficar la línea para la ola actual
    plt.plot(labels, HFR, marker='o', linestyle='-', label=wave, color=color[i])
    i = i + 1
    
plt.xlabel('Age group')
plt.xticks(rotation=45)
plt.ylabel('Ratio')
plt.title('HFR Bogotá')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

## Análisis Nacional

In [ ]:
import math 

def wilson_interval(num, tot, z=1.96):
    if tot == 0:
        return 0, 0
    p = num / tot
    denominator = 1 + (z**2) / tot
    center = p + (z**2) / (2*tot)
    margin = z * math.sqrt((p*(1-p)/tot) + (z**2)/(4*(tot**2)))
    lower = (center - margin) / denominator
    upper = (center + margin) / denominator
    return lower, upper

# Seleccionar los datos Colombia
df_col = df_sivigila346.copy()

# Eliminar registros con edad "No Definido" (son 37 para Colombia)
df_col = df_col[df_col['Edad'] != 'No Definido']

# Convertir la columna de edad a valores numéricos
df_col['EdadAños'] = df_col['Edad'].apply(extraer_edad)

# Agrupar edades
bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, np.inf]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df_col['GrupoEdad'] = pd.cut(df_col['EdadAños'], bins=bins, labels=labels, right=True)

# Convertir las fechas a formato 'datetime'
df_col['FechaNotificacion'] = pd.to_datetime(df_col['FechaNotificacion'], format='%Y%m%d')
df_col['FechaDefuncion'] = pd.to_datetime(df_col['FechaDefuncion'], format='%Y%m%d')
df_col['FechaHospitalizacion'] = pd.to_datetime(df_col['FechaHospitalizacion'], format='%Y%m%d')

# Eliminar fechas de hospitalización erróneas
df_col = df_col[df_col['FechaHospitalizacion'] != '19000101']

# Definir quien falleció o no estando hospitalizado
fecha_no_defuncion = pd.Timestamp('1900-01-01')
df_col['FallecioHosp'] = df_col['FechaDefuncion'] != fecha_no_defuncion

# Descartar los casos donde el ID es igual a 1
df_col = df_col[df_col['PersonaBasicaID'] != '1']

# Definir olas COVID-19 (obtenidas de: https://github.com/TRACE-LAC/covid19-waves-bogota/blob/main/waves/outputs/waves.csv)
waves = {
    'Ola 1': ('2020-02-26', '2020-09-25'),
    'Ola 2': ('2020-11-01', '2021-03-01'),
    'Ola 3': ('2021-03-01', '2021-09-14'),
    'Ola 4': ('2021-11-20', '2022-03-24')
}

# Grafica 
plt.figure(figsize=(5, 5))
colors = ['#6a5acd', '#66c2a5', '#4daf4a', '#d95f02']
i = 0

for wave, (inicio, fin) in waves.items():
    df_wave = df_col[(df_col['FechaNotificacion'] >= inicio) & (df_col['FechaNotificacion'] <= fin)]
    
    # Denominador: gente hospitalizada (fallecida o no) por Covid por grupo de edad
    total_registros = df_wave.groupby('GrupoEdad').size()
    
    # Numerador: gente que fallecio hospitalizada por Covid por grupo de edad
    fallecidosHosp = df_wave[df_wave['FallecioHosp']].groupby('GrupoEdad').size()
    
    # Calcular Case Fatality Ratio (CFR)
    HFR = 100*(fallecidosHosp / total_registros).fillna(0)
    
        # Calcular los errores de cada barra usando el intervalo de Wilson
    lower_errors = []
    upper_errors = []
    for grupo in labels:
        num = fallecidosHosp.get(grupo, 0)
        tot = total_registros.get(grupo, 0)
        if tot > 0:
            lower, upper = wilson_interval(num, tot)
            p = num / tot
            # Convertir a porcentaje y calcular el error inferior y superior
            lower_errors.append(100 * (p - lower))
            upper_errors.append(100 * (upper - p))
        else:
            lower_errors.append(0)
            upper_errors.append(0)
    
    # Graficar la línea para la ola actual
    plt.errorbar(labels, HFR, yerr=[lower_errors, upper_errors],
             fmt='o-', color=colors[i], ecolor='black', capsize=5,
             label=wave)
        
#     plt.plot(labels, HFR, marker='o', linestyle='-', label=wave, color=colors[i])
    i = i + 1
    
    print(fallecidosHosp)
    print(total_registros)
    
    # Graficar la línea para la ola actual
#     plt.plot(labels, HFR, marker='o', linestyle='-', label=wave, color=color[i])
    
plt.xlabel('Grupo de edad', fontsize=12)
plt.xticks(rotation=45)
plt.ylabel('Porcentaje', fontsize=12)
plt.title('Riesgo de Muerte | Hospitalización', fontsize=14)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Formatear el eje y para mostrar porcentajes
from matplotlib.ticker import PercentFormatter
plt.gca().yaxis.set_major_formatter(PercentFormatter(decimals=0))

plt.tight_layout()
plt.show()

---
---
---
## Análisis para todos los departamentos

In [ ]:
# Copiar el DataFrame original (para trabajar con todos los departamentos)
df_sivigila346_v2 = df_sivigila346[df_sivigila346['DepartamentoNotificacion'] != 'NO REPORTADO'].copy()

In [ ]:
# https://dgn.isolutions.iso.org/obp/ui#iso:code:3166:CO
df_sivigila346_v2['DepartamentoNotificacion'].replace({'Bogotá, D.C.': 'DC', 
                                                       'Antioquia': 'ANT', 
                                                       'Valle del Cauca': 'VAC', 
                                                       'Cundinamarca': 'CUN', 
                                                       'Santander': 'SAN', 
                                                       'Atlántico': 'ATL', 
                                                       'Boyacá': 'BOY', 
                                                       'Norte de Santander': 'NSA', 
                                                       'Córdoba': 'COR', 
                                                       'Tolima': 'TOL', 
                                                       'Caldas': 'CAL', 
                                                       'Cesar': 'CES', 
                                                       'Meta': 'MET', 
                                                       'Risaralda': 'RIS', 
                                                       'Huila': 'HUI', 
                                                       'Bolívar': 'BOL', 
                                                       'Cauca': 'CAU', 
                                                       'Sucre': 'SUC', 
                                                       'Quindio': 'QUI', 
                                                       'Nariño': 'NAR', 
                                                       'Magdalena': 'MAG', 
                                                       'Casanare': 'CAS', 
                                                       'La Guajira': 'LAG', 
                                                       'Caquetá': 'CAQ', 
                                                       'Putumayo': 'PUT', 
                                                       'Chocó': 'CHO', 
                                                       'Arauca': 'ARA', 
                                                       'Archipiélago de San Andrés, Providencia y Santa Catalina': 'SAP', 
                                                       'Amazonas': 'AMA', 
                                                       'Guaviare': 'GUV', 
                                                       'Guainía': 'GUA', 
                                                       'Vichada': 'VIC', 
                                                       'Vaupés': 'VAU'}, inplace=True)

In [ ]:
# Eliminar registros con edad "No Definido"
df = df_sivigila346_v2[df_sivigila346_v2['Edad'] != 'No Definido'].copy()

In [ ]:
# Convertir la columna de edad a valores numéricos
df['EdadAños'] = df['Edad'].apply(extraer_edad)

In [ ]:
# Agrupar edades
bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, np.inf]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df['GrupoEdad'] = pd.cut(df['EdadAños'], bins=bins, labels=labels, right=True)

In [ ]:
# Convertir las fechas a formato datetime
df['FechaNotificacion'] = pd.to_datetime(df['FechaNotificacion'], format='%Y%m%d')
df['FechaDefuncion'] = pd.to_datetime(df['FechaDefuncion'], format='%Y%m%d')
df['FechaHospitalizacion'] = pd.to_datetime(df['FechaHospitalizacion'], format='%Y%m%d')

In [ ]:
# Eliminar fechas de hospitalización erróneas
df = df[df['FechaHospitalizacion'] != '19000101']

# Definir quien falleció o no estando hospitalizado
fecha_no_defuncion = pd.Timestamp('1900-01-01')
df['FallecioHosp'] = df['FechaDefuncion'] != fecha_no_defuncion

# Descartar los casos donde el ID es igual a 1
df = df[df['PersonaBasicaID'] != '1']

In [ ]:
waves = {
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

In [ ]:
resultados = []

In [ ]:
for wave, (inicio, fin) in waves.items():
    df_wave = df[(df['FechaNotificacion'] >= inicio) & (df['FechaNotificacion'] <= fin)].copy()
    
    # Denominador: gente hospitalizada (fallecida o no) por Covid por grupo de edad y departamento
    total = df_wave.groupby(['DepartamentoNotificacion', 'GrupoEdad']).size().reset_index(name='Total')
    
    # Numerador: gente que fallecio hospitalizada por Covid por grupo de edad y departamento
    fallh = df_wave[df_wave['FallecioHosp']].groupby(['DepartamentoNotificacion', 'GrupoEdad']).size().reset_index(name='FallH')
    
    # Unir ambos resultados y calcular el HCR
    df_merged = pd.merge(total, fallh, on=['DepartamentoNotificacion', 'GrupoEdad'], how='left')
#     df_merged['Hosp'] = df_merged['Hosp'].fillna(0)
    df_merged['HFR'] = df_merged['FallH'] / df_merged['Total']
    df_merged['Wave'] = wave
    resultados.append(df_merged)

# DataFrame con los resultados consolidados
df_result = pd.concat(resultados, ignore_index=True)
df_result['HFR'] = df_result['HFR'].fillna(0)

df_result = df_result.sort_values(by=['DepartamentoNotificacion',"GrupoEdad"])

In [ ]:
df_result

In [ ]:
# Convertir grupos de edad a posiciones numéricas
group_age_labels = labels  
x_dict = {age: i for i, age in enumerate(group_age_labels)}

# Ordenar los departamentos alfabéticamente
departamentos_sorted = sorted(df_result['DepartamentoNotificacion'].unique())
y_dict = {dep: i for i, dep in enumerate(departamentos_sorted)}

# Definir offsets para cada ola (separación horizontal dentro de cada grupo de edad)
wave_offsets = {
    'Wave 1': -0.3,
    'Wave 2': -0.1,
    'Wave 3': 0.1,
    'Wave 4': 0.3
}

wave_colors = {
    'Wave 1': '#6b6ca3',
    'Wave 2': '#87bcbd',
    'Wave 3': '#6f9954',
    'Wave 4': '#b1615c'
}

plt.figure(figsize=(14, 14))
scale_factor = 1000  # Factor para ajustar el tamaño de las burbujas

for wave in waves.keys():
    df_wave = df_result[df_result['Wave'] == wave]
    # grupo de edad + offset según la ola
    x_vals = df_wave['GrupoEdad'].apply(lambda g: x_dict[str(g)] + wave_offsets[wave])
    y_vals = df_wave['DepartamentoNotificacion'].apply(lambda dep: y_dict[dep])
    sizes = df_wave['HFR'] * scale_factor
    
    plt.scatter(x_vals, y_vals, s=sizes, alpha=0.6, c=wave_colors[wave],
                label=wave, edgecolors='k')

# plt.xticks(ticks=list(x_dict.values()), labels=list(x_dict.keys()), rotation=90)
plt.xticks(ticks=list(x_dict.values()), labels=list(x_dict.keys()))
plt.yticks(ticks=list(y_dict.values()), labels=list(y_dict.keys()))
plt.xlabel('Age Group')
plt.ylabel('Department')
plt.title('HFR')
# plt.legend(title='Waves', bbox_to_anchor=(1.1, 0.5))
plt.grid(True, linestyle='--', alpha=0.5)

plt.gca().invert_yaxis()

legend_handles = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=wave_colors[wave],
           markersize=10, markeredgecolor='k', label=wave, alpha=0.5)
    for wave in waves.keys()
]
# plt.legend(handles=legend_handles, title='Waves', bbox_to_anchor=(1.1, 0.5))

# Leyenda de las olas (ya existente)
legend1 = plt.legend(handles=legend_handles, title='Waves', bbox_to_anchor=(1.12, 0.5))
plt.gca().add_artist(legend1)

# Definir valores representativos para HCR (pueden ajustarse según el rango real)
hcr_min = 0.1
hcr_med = 0.3
hcr_max = 0.5

# Crear handles para los tamaños de burbuja (valores ficticios de ejemplo)
size_handles = [
    plt.scatter([], [], s=0.2 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HFR: 0.2'),
    plt.scatter([], [], s=0.4 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HFR: 0.4'),
    plt.scatter([], [], s=0.6 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HFR: 0.6'),
    plt.scatter([], [], s=0.8 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HFR: 0.8'),
    plt.scatter([], [], s=1.0 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HFR: 1.0'),
    plt.scatter([], [], s=0.0 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' ')
]

# Agregar la leyenda de tamaños debajo de la leyenda de las olas
plt.legend(handles=size_handles, title='     Size bubble     ', bbox_to_anchor=(1.0, 0.4),labelspacing = 4)


plt.tight_layout()
plt.show()

In [ ]:
# Convertir olas a posiciones numéricas
wave_labels = list(waves.keys())
x_dict = {wave: i for i, wave in enumerate(wave_labels)}

# Ordenar los departamentos alfabéticamente
departamentos_sorted = sorted(df_result['DepartamentoNotificacion'].unique())
y_dict = {dep: i for i, dep in enumerate(departamentos_sorted)}

# Definir offsets para cada grupo de edad (separación horizontal dentro de cada ola)

age_offsets = {age: i * 0.1 - 0.4 for i, age in enumerate(labels)}

age_colors = {
    age: plt.cm.viridis(i / len(labels)) for i, age in enumerate(labels)
}

plt.figure(figsize=(12, 15))
scale_factor = 1200  # Factor para ajustar el tamaño de las burbujas

for age in labels:
    df_age = df_result[df_result['GrupoEdad'] == age]
    # ola + offset según el grupo de edad
    x_vals = df_age['Wave'].apply(lambda w: x_dict[w] + age_offsets[age])
    y_vals = df_age['DepartamentoNotificacion'].apply(lambda dep: y_dict[dep])
    sizes = df_age['HFR'] * scale_factor
    
    plt.scatter(x_vals, y_vals, s=sizes, alpha=0.6, c=[age_colors[age]] * len(x_vals),
                label=age if age not in plt.gca().get_legend_handles_labels()[1] else "", edgecolors='k')

plt.xticks(ticks=list(x_dict.values()), labels=list(x_dict.keys()))
plt.yticks(ticks=list(y_dict.values()), labels=list(y_dict.keys()))
plt.xlabel('Wave')
plt.ylabel('Department')
plt.title('HFR')
plt.grid(True, linestyle='--', alpha=0.5)
plt.gca().invert_yaxis()

# Leyenda de los grupos de edad
legend_handles = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=age_colors[age],
           markersize=10, markeredgecolor='k', label=age, alpha=0.5)
    for age in labels
]


legend1 =plt.legend(handles=legend_handles, title='Age Groups', bbox_to_anchor=(1.12, 0.8))
plt.gca().add_artist(legend1)

# Definir valores representativos para HCR
# size_handles = [
#     plt.scatter([], [], s=0.2 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HCR: 0.2'),
#     plt.scatter([], [], s=0.4 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HCR: 0.4'),
#     plt.scatter([], [], s=0.6 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HCR: 0.6'),
#     plt.scatter([], [], s=0.8 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HCR: 0.8'),
#     plt.scatter([], [], s=1.0 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HCR: 1.0')
# ]
size_handles = [
    plt.scatter([], [], s=val * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=f' HFR: {val}')
    for val in [0.2, 0.4, 0.6, 0.8, 1.0]
]
size_handles.append(plt.scatter([], [], s=0, color='gray', alpha=0.6, edgecolors='k', label=' '))

plt.legend(handles=size_handles, title='     Size bubble     ', bbox_to_anchor=(1.0, 0.6), labelspacing=4)

plt.tight_layout()
plt.show()


In [ ]:
# Heatmap
# ------------------------------

waves_list = list(waves.keys())
n_waves = len(waves_list)

fig, axs = plt.subplots(2, 2, figsize=(14, 14))
axs = axs.flatten()

for i, wave in enumerate(waves_list):
    df_wave = df_result[df_result['Wave'] == wave].copy()
    heatmap_data = df_wave.pivot(index='DepartamentoNotificacion', columns='GrupoEdad', values='HFR')
    heatmap_data = heatmap_data.sort_index()  
    sns.heatmap(heatmap_data, ax=axs[i], cmap='viridis', annot=True, fmt=".2f", cbar_kws={'label': 'HFR'})
    axs[i].set_title(f'HFR  - {wave}')
    axs[i].set_xlabel('Age Group')
    axs[i].set_ylabel('Department')

plt.tight_layout()
plt.show()

In [ ]:
waves_list = list(waves.keys())

# Grafica
fig, axs = plt.subplots(2, 2, figsize=(14, 16))
axs = axs.flatten()

# Factor de escala burbujas
scale_factor = 1000

for i, wave in enumerate(waves_list):
    df_wave = df_result[df_result['Wave'] == wave]
    scatter = axs[i].scatter(
        x=df_wave['GrupoEdad'].astype(str),
        y=df_wave['DepartamentoNotificacion'],
        s=df_wave['HFR'] * scale_factor,
        alpha=0.6,
        c=df_wave['HFR'],
        cmap='viridis'
    )
    axs[i].set_title(f'HFR - {wave}')
    axs[i].set_xlabel('Age Group')
    axs[i].set_ylabel('Department')
    
    axs[i].tick_params(axis='x', labelrotation=45)
    
    axs[i].invert_yaxis()
    
    cbar = plt.colorbar(scatter, ax=axs[i])
#     cbar.set_label('HCR', rotation=0, y=1.0)

plt.tight_layout()
plt.show()

---
---
---
## Análisis para todos los departamentos POR REGION

In [ ]:
# Copiar el DataFrame original (para trabajar con todos los departamentos)
df_sivigila346_v2 = df_sivigila346[df_sivigila346['DepartamentoNotificacion'] != 'NO REPORTADO'].copy()

In [ ]:
# https://dgn.isolutions.iso.org/obp/ui#iso:code:3166:CO
df_sivigila346_v2['DepartamentoNotificacion'].replace({'Bogotá, D.C.': 'DC', 
                                                       'Antioquia': 'ANT', 
                                                       'Valle del Cauca': 'VAC', 
                                                       'Cundinamarca': 'CUN', 
                                                       'Santander': 'SAN', 
                                                       'Atlántico': 'ATL', 
                                                       'Boyacá': 'BOY', 
                                                       'Norte de Santander': 'NSA', 
                                                       'Córdoba': 'COR', 
                                                       'Tolima': 'TOL', 
                                                       'Caldas': 'CAL', 
                                                       'Cesar': 'CES', 
                                                       'Meta': 'MET', 
                                                       'Risaralda': 'RIS', 
                                                       'Huila': 'HUI', 
                                                       'Bolívar': 'BOL', 
                                                       'Cauca': 'CAU', 
                                                       'Sucre': 'SUC', 
                                                       'Quindio': 'QUI', 
                                                       'Nariño': 'NAR', 
                                                       'Magdalena': 'MAG', 
                                                       'Casanare': 'CAS', 
                                                       'La Guajira': 'LAG', 
                                                       'Caquetá': 'CAQ', 
                                                       'Putumayo': 'PUT', 
                                                       'Chocó': 'CHO', 
                                                       'Arauca': 'ARA', 
                                                       'Archipiélago de San Andrés, Providencia y Santa Catalina': 'SAP', 
                                                       'Amazonas': 'AMA', 
                                                       'Guaviare': 'GUV', 
                                                       'Guainía': 'GUA', 
                                                       'Vichada': 'VIC', 
                                                       'Vaupés': 'VAU'}, inplace=True)

In [ ]:
def classify_region(department):
    region_map = {
        "Amazónica": ["AMA", "CAQ", "GUA", "GUV", "PUT", "VAU"],
        "Andina": ["ANT", "BOY", "CAL", "CUN", "DC", "HUI", "NSA", "QUI", "RIS", "SAN", "TOL"],
        "Pacífica": ["VAC", "CHO", "CAU", "NAR"],
        "Caribe e Insular": ["ATL", "BOL", "CES", "COR", "LAG", "MAG", "SUC", "SAP"],
        "Orinoquía": ["ARA", "CAS", "MET", "VIC"]
    }
    
    for region, departments in region_map.items():
        if department in departments:
            return region
    return "Desconocido"  # En caso de que haya códigos no contemplados

# Aplicar la función a la columna DepartamentoNotificacion
df_sivigila346_v2["Region"] = df_sivigila346_v2["DepartamentoNotificacion"].apply(classify_region)

In [ ]:
# Eliminar registros con edad "No Definido"
df = df_sivigila346_v2[df_sivigila346_v2['Edad'] != 'No Definido'].copy()

# Convertir la columna de edad a valores numéricos
df['EdadAños'] = df['Edad'].apply(extraer_edad)

# Agrupar edades
bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, np.inf]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df['GrupoEdad'] = pd.cut(df['EdadAños'], bins=bins, labels=labels, right=True)

In [ ]:
# Convertir las fechas a formato datetime
df['FechaNotificacion'] = pd.to_datetime(df['FechaNotificacion'], format='%Y%m%d')
df['FechaDefuncion'] = pd.to_datetime(df['FechaDefuncion'], format='%Y%m%d')
df['FechaHospitalizacion'] = pd.to_datetime(df['FechaHospitalizacion'], format='%Y%m%d')

# Eliminar fechas de hospitalización erróneas
df = df[df['FechaHospitalizacion'] != '19000101']

# Definir quien falleció o no estando hospitalizado
fecha_no_defuncion = pd.Timestamp('1900-01-01')
df['FallecioHosp'] = df['FechaDefuncion'] != fecha_no_defuncion

# Descartar los casos donde el ID es igual a 1
df = df[df['PersonaBasicaID'] != '1']

waves = {
    'Ola 1': ('2020-02-26', '2020-09-25'),
    'Ola 2': ('2020-11-01', '2021-03-01'),
    'Ola 3': ('2021-03-01', '2021-09-14'),
    'Ola 4': ('2021-11-20', '2022-03-24')
}

In [ ]:
resultados = []

In [ ]:
for wave, (inicio, fin) in waves.items():
    df_wave = df[(df['FechaNotificacion'] >= inicio) & (df['FechaNotificacion'] <= fin)].copy()
    
    # Denominador: gente hospitalizada (fallecida o no) por Covid por grupo de edad y departamento
    total = df_wave.groupby(['Region', 'GrupoEdad']).size().reset_index(name='Total')
    
    # Numerador: gente que fallecio hospitalizada por Covid por grupo de edad y departamento
    fallh = df_wave[df_wave['FallecioHosp']].groupby(['Region', 'GrupoEdad']).size().reset_index(name='FallH')
    
    # Unir ambos resultados y calcular el HCR
    df_merged = pd.merge(total, fallh, on=['Region', 'GrupoEdad'], how='left')
#     df_merged['Hosp'] = df_merged['Hosp'].fillna(0)
    df_merged['HFR'] = df_merged['FallH'] / df_merged['Total']
    df_merged['Wave'] = wave
    resultados.append(df_merged)

# DataFrame con los resultados consolidados
df_result = pd.concat(resultados, ignore_index=True)
df_result['HFR'] = df_result['HFR'].fillna(0)

df_result = df_result.sort_values(by=['Region',"GrupoEdad"])

In [ ]:
waves_list = list(waves.keys())

# Grafica
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
axs = axs.flatten()

# Factor de escala burbujas
scale_factor = 1200

for i, wave in enumerate(waves_list):
    df_wave = df_result[df_result['Wave'] == wave]
    scatter = axs[i].scatter(
        x=df_wave['GrupoEdad'].astype(str),
        y=df_wave['Region'],
        s=df_wave['HFR'] * scale_factor,
        alpha=0.6,
        c=df_wave['HFR'],
        cmap='viridis'
    )
    axs[i].set_title(f'HFR - {wave}')
    axs[i].set_xlabel('Age Group')
    axs[i].set_ylabel('Region')
    
    axs[i].tick_params(axis='x', labelrotation=45)
    
    axs[i].invert_yaxis()
    
    cbar = plt.colorbar(scatter, ax=axs[i])
    cbar.set_label('HFR')

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap
# ------------------------------

waves_list = list(waves.keys())
n_waves = len(waves_list)

fig, axs = plt.subplots(2, 2, figsize=(12, 8))
axs = axs.flatten()

for i, wave in enumerate(waves_list):
    df_wave = df_result[df_result['Wave'] == wave].copy()
    heatmap_data = df_wave.pivot(index='Region', columns='GrupoEdad', values='HFR')
    heatmap_data = heatmap_data.sort_index()  
    sns.heatmap(heatmap_data, ax=axs[i], cmap='viridis', annot=True, fmt=".2f", cbar_kws={'label': 'HFR'})
    axs[i].set_title(f'HFR  - {wave}')
    axs[i].set_xlabel('Age Group')
    axs[i].set_ylabel('Region')

plt.tight_layout()
plt.show()


In [ ]:
# Convertir grupos de edad a posiciones numéricas
group_age_labels = labels  
x_dict = {age: i for i, age in enumerate(group_age_labels)}

# Ordenar los departamentos alfabéticamente
departamentos_sorted = sorted(df_result['Region'].unique())
y_dict = {dep: i for i, dep in enumerate(departamentos_sorted)}

# Definir offsets para cada ola (separación horizontal dentro de cada grupo de edad)
wave_offsets = {
    'Wave 1': -0.3,
    'Wave 2': -0.1,
    'Wave 3': 0.1,
    'Wave 4': 0.3
}

wave_colors = {
    'Wave 1': '#6b6ca3',
    'Wave 2': '#87bcbd',
    'Wave 3': '#6f9954',
    'Wave 4': '#b1615c'
}

plt.figure(figsize=(10, 5))
scale_factor = 500  # Factor para ajustar el tamaño de las burbujas

for wave in waves.keys():
    df_wave = df_result[df_result['Wave'] == wave]
    # grupo de edad + offset según la ola
    x_vals = df_wave['GrupoEdad'].apply(lambda g: x_dict[str(g)] + wave_offsets[wave])
    y_vals = df_wave['Region'].apply(lambda dep: y_dict[dep])
    sizes = df_wave['HFR'] * scale_factor
    
    plt.scatter(x_vals, y_vals, s=sizes, alpha=0.6, c=wave_colors[wave],
                label=wave, edgecolors='k')

# plt.xticks(ticks=list(x_dict.values()), labels=list(x_dict.keys()), rotation=90)
plt.xticks(ticks=list(x_dict.values()), labels=list(x_dict.keys()))
plt.yticks(ticks=list(y_dict.values()), labels=list(y_dict.keys()))
plt.xlabel('Age Group')
plt.ylabel('Region')
plt.title('HFR')
# plt.legend(title='Waves', bbox_to_anchor=(1.1, 0.5))
plt.grid(True, linestyle='--', alpha=0.5)

plt.gca().invert_yaxis()

legend_handles = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=wave_colors[wave],
           markersize=10, markeredgecolor='k', label=wave, alpha=0.5)
    for wave in waves.keys()
]

# Leyenda de las olas (ya existente)
legend1 = plt.legend(handles=legend_handles, title='Waves', bbox_to_anchor=(1.2, 0.9))
plt.gca().add_artist(legend1)

# Definir valores representativos para HCR (pueden ajustarse según el rango real)
hcr_min = 0.1
hcr_med = 0.3
hcr_max = df_result['HFR'].max()

# Crear handles para los tamaños de burbuja (valores ficticios de ejemplo)
size_handles = [
    plt.scatter([], [], s=0.20 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HFR: 0.2'),
    plt.scatter([], [], s=0.40 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HFR: 0.4'),
    plt.scatter([], [], s=0.60 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HFR: 0.6'),
    plt.scatter([], [], s=hcr_max * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=f'HFR: {hcr_max:.2f}'),
    plt.scatter([], [], s=0.0 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' ')
]

# Agregar la leyenda de tamaños debajo de la leyenda de las olas
plt.legend(handles=size_handles, title='     Size bubble     ', bbox_to_anchor=(1.01, 0.6),labelspacing =3)


plt.tight_layout()
plt.show()

In [ ]:
# Convertir grupos de edad a posiciones numéricas para el eje Y
group_age_labels = labels  
y_dict = {age: i for i, age in enumerate(group_age_labels)}

# Ordenar las regiones alfabéticamente para el eje X
regions_sorted = sorted(df_result['Region'].unique())
x_dict = {region: i for i, region in enumerate(regions_sorted)}

# Definir offsets para cada ola (separación horizontal dentro de cada región)
wave_offsets = {
    'Wave 1': -0.3,
    'Wave 2': -0.1,
    'Wave 3': 0.1,
    'Wave 4': 0.3
}

wave_colors = {
    'Wave 1': '#6b6ca3',
    'Wave 2': '#87bcbd',
    'Wave 3': '#6f9954',
    'Wave 4': '#b1615c'
}

plt.figure(figsize=(10, 5))
scale_factor = 500  # Factor para ajustar el tamaño de las burbujas

for wave in waves.keys():
    df_wave = df_result[df_result['Wave'] == wave]
    # Eje X: la región + offset según la ola para separar horizontalmente
    # Eje Y: el grupo de edad
    x_vals = df_wave['Region'].apply(lambda r: x_dict[r] + wave_offsets[wave])
    y_vals = df_wave['GrupoEdad'].apply(lambda g: y_dict[str(g)])
    sizes = df_wave['HFR'] * scale_factor

    plt.scatter(x_vals, y_vals, s=sizes, alpha=0.6, c=wave_colors[wave],
                label=wave, edgecolors='k')

plt.xticks(ticks=list(x_dict.values()), labels=list(x_dict.keys()))
plt.yticks(ticks=list(y_dict.values()), labels=list(y_dict.keys()))
plt.xlabel('Region')
plt.ylabel('Age Group')
plt.title('HFR')
plt.grid(True, linestyle='--', alpha=0.5)

plt.gca().invert_yaxis()

# Leyenda de las olas
legend_handles = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=wave_colors[wave],
           markersize=10, markeredgecolor='k', label=wave, alpha=0.5)
    for wave in waves.keys()
]
legend1 = plt.legend(handles=legend_handles, title='Waves', bbox_to_anchor=(1.2, 0.9))
plt.gca().add_artist(legend1)

# Leyenda para los tamaños de burbuja
hcr_min = 0.1
hcr_med = 0.3
hcr_max = df_result['HFR'].max()

size_handles = [
    plt.scatter([], [], s=0.20 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HFR: 0.2'),
    plt.scatter([], [], s=0.40 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HFR: 0.4'),
    plt.scatter([], [], s=0.60 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HFR: 0.6'),
    plt.scatter([], [], s=hcr_max * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=f'HFR: {hcr_max:.2f}'),
    plt.scatter([], [], s=0.0 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' ')
]

plt.legend(handles=size_handles, title='     Size bubble     ', bbox_to_anchor=(1.01, 0.6), labelspacing=3)

plt.tight_layout()
plt.show()

In [ ]:
# Convertir olas a posiciones numéricas
wave_labels = list(waves.keys())
x_dict = {wave: i for i, wave in enumerate(wave_labels)}

# Ordenar los departamentos alfabéticamente
departamentos_sorted = sorted(df_result['Region'].unique())
y_dict = {dep: i for i, dep in enumerate(departamentos_sorted)}

# Definir offsets para cada grupo de edad (separación horizontal dentro de cada ola)

age_offsets = {age: i * 0.1 - 0.4 for i, age in enumerate(labels)}

age_colors = {
    age: plt.cm.viridis(i / len(labels)) for i, age in enumerate(labels)
}

plt.figure(figsize=(10, 6))
scale_factor = 1000  # Factor para ajustar el tamaño de las burbujas

for age in labels:
    df_age = df_result[df_result['GrupoEdad'] == age]
    # ola + offset según el grupo de edad
    x_vals = df_age['Wave'].apply(lambda w: x_dict[w] + age_offsets[age])
    y_vals = df_age['Region'].apply(lambda dep: y_dict[dep])
    sizes = df_age['HFR'] * scale_factor
    
    plt.scatter(x_vals, y_vals, s=sizes, alpha=0.6, c=[age_colors[age]] * len(x_vals),
                label=age if age not in plt.gca().get_legend_handles_labels()[1] else "", edgecolors='k')

plt.xticks(ticks=list(x_dict.values()), labels=list(x_dict.keys()))
plt.yticks(ticks=list(y_dict.values()), labels=list(y_dict.keys()))
plt.xlabel('Wave')
plt.ylabel('Region')
plt.title('HFR')
plt.grid(True, linestyle='--', alpha=0.5)
plt.gca().invert_yaxis()

# Leyenda de los grupos de edad
legend_handles = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=age_colors[age],
           markersize=10, markeredgecolor='k', label=age, alpha=0.5)
    for age in labels
]


legend1 =plt.legend(handles=legend_handles, title='Age Groups', bbox_to_anchor=(1.16, 1))
plt.gca().add_artist(legend1)

# Definir valores representativos para HCR
# size_handles = [
#     plt.scatter([], [], s=0.2 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HCR: 0.2'),
#     plt.scatter([], [], s=0.4 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HCR: 0.4'),
#     plt.scatter([], [], s=0.6 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HCR: 0.6'),
#     plt.scatter([], [], s=0.8 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HCR: 0.8'),
#     plt.scatter([], [], s=1.0 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' HCR: 1.0')
# ]
size_handles = [
    plt.scatter([], [], s=0.10 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' 0.1'),
    plt.scatter([], [], s=0.20 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' 0.2'),
    plt.scatter([], [], s=0.30 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' 0.3'),
    plt.scatter([], [], s=hcr_max * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=f'{hcr_max:.2f}'),
    plt.scatter([], [], s=0.0 * scale_factor, color='gray', alpha=0.6, edgecolors='k', label=' ')
]


plt.legend(handles=size_handles, title='   Size bubble  ', bbox_to_anchor=(1.005, 0.5), labelspacing=2)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, (wave, (inicio, fin)) in enumerate(waves.items()):
    df_wave = df[(df['FechaNotificacion'] >= inicio) & (df['FechaNotificacion'] <= fin)].copy()
    
    total_df = df_wave.groupby(['Region', 'GrupoEdad']).size().reset_index(name='total')
    fallh = df_wave[df_wave['FallecioHosp']].groupby(['Region', 'GrupoEdad']).size().reset_index(name='FallH')
    
    merged_df = pd.merge(total_df, fallh, on=['Region', 'GrupoEdad'], how='left')
    merged_df['FallH'] = merged_df['FallH'].fillna(0)
    merged_df['HFR'] = merged_df['FallH'] / merged_df['total']
    
    # Pivot: filas = GrupoEdad, columnas = Departamento
    pivot_df = merged_df.pivot(index='GrupoEdad', columns='Region', values='HFR')
    pivot_df = pivot_df.reindex(labels)  # Asegurar el orden de los grupos de edad
    
    # Graficar barras agrupadas (usar stacked=True para barras apiladas)
    pivot_df.plot(kind='bar', ax=axes[i], width=0.8, stacked=False)
    axes[i].set_title(f'{wave}')
    axes[i].set_xlabel('Age Group')
    axes[i].set_ylabel('HFR')
    axes[i].legend(title='Region', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, (wave, (inicio, fin)) in enumerate(waves.items()):
    df_wave = df[(df['FechaNotificacion'] >= inicio) & (df['FechaNotificacion'] <= fin)].copy()
    
    total_df = df_wave.groupby(['Region', 'GrupoEdad']).size().reset_index(name='total')
    fallh = df_wave[df_wave['FallecioHosp']].groupby(['Region', 'GrupoEdad']).size().reset_index(name='FallH')
    
    merged_df = pd.merge(total_df, fallh, on=['Region', 'GrupoEdad'], how='left')
    merged_df['FallH'] = merged_df['FallH'].fillna(0)
    merged_df['HFR'] = merged_df['FallH'] / merged_df['total']
    
    # Pivot: filas = GrupoEdad, columnas = Region
    pivot_df = merged_df.pivot(index='GrupoEdad', columns='Region', values='HFR')
    pivot_df = pivot_df.reindex(labels)  # Asegurar el orden de los grupos de edad
    
    # Graficar líneas para cada Región con marcadores
    pivot_df.plot(ax=axes[i], marker='o', linestyle='-', linewidth=2)
    axes[i].set_title(f'{wave}')
    axes[i].set_xlabel('Age Group')
    axes[i].set_ylabel('HFR')
    axes[i].legend(title='Region', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

fig, axes = plt.subplots(2, 2, figsize=(11, 11))
axes = axes.flatten()

first_handles, first_labels = None, None  # Para capturar los handles y labels de la leyenda

for i, (wave, (inicio, fin)) in enumerate(waves.items()):
    df_wave = df[(df['FechaNotificacion'] >= inicio) & (df['FechaNotificacion'] <= fin)].copy()
    
    total_df = df_wave.groupby(['Region', 'GrupoEdad']).size().reset_index(name='total')
    fallh = df_wave[df_wave['FallecioHosp']].groupby(['Region', 'GrupoEdad']).size().reset_index(name='FallH')
    
    merged_df = pd.merge(total_df, fallh, on=['Region', 'GrupoEdad'], how='left')
    merged_df['FallH'] = merged_df['FallH'].fillna(0)
    merged_df['HFR'] = merged_df['FallH'] / merged_df['total']
    
    # Pivot: filas = GrupoEdad, columnas = Region
    pivot_df = merged_df.pivot(index='GrupoEdad', columns='Region', values='HFR')
    pivot_df = pivot_df.reindex(labels)  # Asegurar el orden de los grupos de edad
    
    # Graficar líneas para cada Región con marcadores
    ax = axes[i]
    
#     # Para cada Región, calcular error (intervalo de Wilson) y graficar con errorbar
#     for j, region in enumerate(pivot_df.columns):
#         hfr_series = pivot_df[region]
#         lower_errors = []
#         upper_errors = []
#         # Calcular los errores por cada grupo de edad en el orden de 'labels'
#         for grupo in labels:
#             fila = merged_df[(merged_df['Region'] == region) & (merged_df['GrupoEdad'] == grupo)]
#             if not fila.empty:
#                 tot = fila['total'].values[0]
#                 num = fila['FallH'].values[0]
#                 lower, upper = wilson_interval(num, tot)
#                 p = num / tot
#                 lower_errors.append(p - lower)
#                 upper_errors.append(upper - p)
#             else:
#                 lower_errors.append(0)
#                 upper_errors.append(0)
        
#         color = colors[j % len(colors)]
#         ax.errorbar(labels, hfr_series, yerr=[lower_errors, upper_errors],
#                     fmt='o-', ecolor='black', capsize=5,
#                     markersize=5, linewidth=2, label=region)
    
    
    
    lines = pivot_df.plot(ax=ax, marker='o', linestyle='-', linewidth=2)
    ax.set_title(f'{wave}', fontsize=14)
    ax.set_xlabel('Grupo de edad', fontsize=13)
    ax.set_ylabel('Riesgo de Muerte | Hospitalización', fontsize=14)
    
    # Configurar eje Y: límite de 0 a 80% y formato porcentual
    ax.set_ylim(0, 0.8)
    ax.yaxis.set_major_formatter(PercentFormatter(1))

    
    ax.tick_params(axis='x', labelsize=12)  # Tamaño de fuente para etiquetas eje X
    plt.setp(ax.get_xticklabels(), rotation=45)
    
    
    ax.tick_params(axis='y', labelsize=12)
    ax.grid(True, linestyle='--', alpha=0.6)
    
    # Remover la leyenda individual para usar una única leyenda global
    if ax.get_legend() is not None:
        ax.legend_.remove()
    
    # Capturar los handles y labels de la primera iteración para la leyenda global
    if first_handles is None:
        first_handles, first_labels = ax.get_legend_handles_labels()

# Crear una única leyenda para todas las gráficas
fig.legend(first_handles, first_labels, title='Region',
           bbox_to_anchor=(1.01, 0.97), loc='upper left', fontsize=12, title_fontsize=12)

plt.tight_layout()
plt.show()
